# Object detection pipeline (torchgeo + torchvision)

Downloads the tiles for the desired county, runs training, and runs inference.


Create conda environment that includes gdal and all dependencies.  Save as a kernel.

*Only need to do this once.  Once the conda environment and kernel are created, select "Python (vrt)" as the Jupyter kernel*

In [ ]:

# # remove any corrupt cached pkgs and temporary downloads
# !conda clean -y --all

# # ensure channel priority avoids mixing variants
# !conda config --set channel_priority strict


# # create env with python, gdal, rasterio
# !conda create -y -n vrt -c conda-forge python=3.11 gdal rasterio

# # install pytorch with cuda support
# !conda run -n vrt conda install -y -c pytorch -c nvidia pytorch torchvision pytorch-cuda=12.1

# # install mamba
# !conda install -n base -c conda-forge -y mamba

# # install packages
# !mamba install -n vrt -c conda-forge -y \
#   tqdm \
#   geopandas \
#   shapely \
#   fiona \
#   pyproj \
#   rtree \
#   scikit-image \
#   opencv

# # register the kernel
# !conda run -n vrt python -m pip install ipykernel
# !conda run -n vrt python -m ipykernel install --user --name vrt --display-name "Python (vrt)"


Will remove 156 (1.15 GB) tarball(s).
Will remove 1 index cache(s).
Will remove 75 (2.63 GB) package(s).
There are no tempfile(s) to remove.
There are no logfile(s) to remove.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/sagemaker-user/.conda/envs/vrt

  added / updated specs:
    - gdal
    - python=3.11
    - rasterio


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |           20_gnu          28 KB  conda-forge
    ld_impl_linux-64-2.45.1    |default_hbd61a6d_101         709 KB  conda-forge
    libblas-3.11.0             |5_h4a7cf45_openblas          18 KB  conda-forge
    libcblas-3.11.0            |5_h0358290_openblas          18 KB  conda-forge
    libgcc-ng-15.2.0           |      h69a702a_18          27 KB  conda-forge
    libgfortran-15.2.0         |      h69a702a_18          

Identify root directory

In [1]:
from pathlib import Path, PurePosixPath
import sys
project_root = Path.cwd().resolve().parent  # notebooks → root
sys.path.insert(0, str(project_root / "src"))


Download tiles

In [ ]:
from utils.indiana_cogs import download_6in_tiles

summary = download_6in_tiles("Benton", max_workers=24)
summary


Benton 06in tiles (2023): 100%|██████████| 1999/1999 [18:47<00:00,  1.77file/s, dl=1999, res=0, skip=0, fail=0]


{'county': 'Benton',
 'year': 2023,
 'output_dir': '/home/sagemaker-user/runoff_classifier/data/counties/Benton/tiles',
 'total': 1999,
 'downloaded': 1999,
 'resumed': 0,
 'skipped': 0,
 'failed': 0}

Convert input tif to correct format (not needed if starting from tif tiles)

In [ ]:

# # type: ignore
# !gdal_translate -co TILED=YES -co COMPRESS=NONE -co BIGTIFF=YES -co BLOCKXSIZE=512 -co BLOCKYSIZE=512 \
#     ../data/bwe/Benton_Mosaic_Clip.tif ../data/bwe/Benton_Mosaic_Clip_prepared.tif


/bin/bash: line 1: gdal_translate: command not found


Run training and inference

In [2]:

# type: ignore

!python ../scripts/train.py \
    --task instance_seg \
    --raster_path ../data/counties/Benton/tiles/ \
    --normalize imagenet \
    --labels_path ../data/counties/Benton/BentonDetect.shp \
    --out_dir ../outputs/counties/Benton \
    --tile_size 512 \
    --epochs 12 \
    --batch_size 2 \


!python ../scripts/inference.py \
    --task instance_seg \
    --raster_path ../data/counties/Benton/tiles/ \
    --normalize imagenet \
    --checkpoint ../outputs/counties/Benton/model_final.pth \
    --out_vector ../outputs/counties/Benton/detections.gpkg \
    --tile_size 1024 \
    --stride 512 \
    --score_thresh 0.5 \
    --nms_iou_thresh 0.5 \
    --device cuda

[info] using labels: ../data/counties/Benton/BentonDetect_reproj.shp
epoch 1/12:   0%|                                       | 0/402 [00:00<?, ?it/s]/home/sagemaker-user/.conda/envs/vrt/lib/python3.11/site-packages/torch/utils/data/dataloader.py:1118: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
epoch 1/12:  18%|█████                       | 73/402 [16:04<1:12:30, 13.22s/it]^C
2026-02-18 21:48:30,721 INFO [debug] starting inference
Traceback (most recent call last):
  File "/home/sagemaker-user/runoff_classifier/notebooks/../scripts/inference.py", line 331, in <module>
    main()
  File "/home/sagemaker-user/runoff_classifier/notebooks/../scripts/inference.py", line 148, in main
    classes, task_from_ckpt = load_classes_and_task_from_checkpoint(args.checkpoint)
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/sagemaker-user/runoff_clas